In [ ]:
import numpy as np
import pandas as pd
import pickle as pkl
import matplotlib.pyplot as plt
import seaborn as sns
import joblib as jb

In [ ]:
df_deliveries = pd.read_csv('/content/deliveries.csv')
df_matches = pd.read_csv('/content/matches.csv')

In [ ]:
df_deliveries.head()

,match_id,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,...,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed,dismissal_kind,fielder
0,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,1,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
1,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,2,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
2,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,3,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,4,0,4,NaN,NaN,NaN
3,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,4,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
4,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,5,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,2,2,NaN,NaN,NaN


In [ ]:
df_matches.head()

,id,Season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3
0,1,IPL-2017,Hyderabad,05-04-2017,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Sunrisers Hyderabad,35,0,Yuvraj Singh,"Rajiv Gandhi International Stadium, Uppal",AY Dandekar,NJ Llong,NaN
1,2,IPL-2017,Pune,06-04-2017,Mumbai Indians,Rising Pune Supergiant,Rising Pune Supergiant,field,normal,0,Rising Pune Supergiant,0,7,SPD Smith,Maharashtra Cricket Association Stadium,A Nand Kishore,S Ravi,NaN
2,3,IPL-2017,Rajkot,07-04-2017,Gujarat Lions,Kolkata Knight Riders,Kolkata Knight Riders,field,normal,0,Kolkata Knight Riders,0,10,CA Lynn,Saurashtra Cricket Association Stadium,Nitin Menon,CK Nandan,NaN
3,4,IPL-2017,Indore,08-04-2017,Rising Pune Supergiant,Kings XI Punjab,Kings XI Punjab,field,normal,0,Kings XI Punjab,0,6,GJ Maxwell,Holkar Cricket Stadium,AK Chaudhary,C Shamshuddin,NaN
4,5,IPL-2017,Bangalore,08-04-2017,Royal Challengers Bangalore,Delhi Daredevils,Royal Challengers Bangalore,bat,normal,0,Royal Challengers Bangalore,15,0,KM Jadhav,M Chinnaswamy Stadium,NaN,NaN,NaN


In [ ]:
total_runs_df = df_deliveries.groupby(['match_id', 'inning']).sum()['total_runs'].reset_index()

In [ ]:
total_runs_df = total_runs_df[total_runs_df['inning'] == 1]

In [ ]:
total_runs_df['total_runs'] = total_runs_df['total_runs'] + 1

In [ ]:
total_runs_df = total_runs_df.rename(columns = {'total_runs' : 'target'})

In [ ]:
total_runs_df

,match_id,inning,target
0,1,1,208
2,2,1,185
4,3,1,184
6,4,1,164
8,5,1,158
...,...,...,...
1518,11347,1,144
1520,11412,1,137
1522,11413,1,172
1524,11414,1,156


In [ ]:
match_df = df_matches.merge(total_runs_df[['match_id', 'target']], left_on = 'id', right_on = 'match_id')

In [ ]:
match_df.head()

,id,Season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3,match_id,target
0,1,IPL-2017,Hyderabad,05-04-2017,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Sunrisers Hyderabad,35,0,Yuvraj Singh,"Rajiv Gandhi International Stadium, Uppal",AY Dandekar,NJ Llong,NaN,1,208
1,2,IPL-2017,Pune,06-04-2017,Mumbai Indians,Rising Pune Supergiant,Rising Pune Supergiant,field,normal,0,Rising Pune Supergiant,0,7,SPD Smith,Maharashtra Cricket Association Stadium,A Nand Kishore,S Ravi,NaN,2,185
2,3,IPL-2017,Rajkot,07-04-2017,Gujarat Lions,Kolkata Knight Riders,Kolkata Knight Riders,field,normal,0,Kolkata Knight Riders,0,10,CA Lynn,Saurashtra Cricket Association Stadium,Nitin Menon,CK Nandan,NaN,3,184
3,4,IPL-2017,Indore,08-04-2017,Rising Pune Supergiant,Kings XI Punjab,Kings XI Punjab,field,normal,0,Kings XI Punjab,0,6,GJ Maxwell,Holkar Cricket Stadium,AK Chaudhary,C Shamshuddin,NaN,4,164
4,5,IPL-2017,Bangalore,08-04-2017,Royal Challengers Bangalore,Delhi Daredevils,Royal Challengers Bangalore,bat,normal,0,Royal Challengers Bangalore,15,0,KM Jadhav,M Chinnaswamy Stadium,NaN,NaN,NaN,5,158


In [ ]:
match_df['team1'].unique()

array(['Sunrisers Hyderabad', 'Mumbai Indians', 'Gujarat Lions',
       'Rising Pune Supergiant', 'Royal Challengers Bangalore',
       'Kolkata Knight Riders', 'Delhi Daredevils', 'Kings XI Punjab',
       'Chennai Super Kings', 'Rajasthan Royals', 'Deccan Chargers',
       'Kochi Tuskers Kerala', 'Pune Warriors', 'Rising Pune Supergiants',
       'Delhi Capitals'], dtype=object)

In [ ]:
match_df['team1'] = match_df['team1'].str.replace('Delhi Daredevils' , 'Delhi Capitals')
match_df['team2'] = match_df['team2'].str.replace('Delhi Daredevils' , 'Delhi Capitals')

match_df['team1'] = match_df['team1'].str.replace('Deccan Chargers' , 'Sunrisers Hyderabad')
match_df['team2'] = match_df['team2'].str.replace('Deccan Chargers' , 'Sunrisers Hyderabad')

In [ ]:
teams = [
    'Sunrisers Hyderabad',
    'Mumbai Indians',
    'Chennai Super Kings',
    'Royal Challengers Bangalore',
    'Kings XI Punjab',
    'Delhi Capitals',
    'Kolkata Knight Riders',
    'Rajasthan Royals'
]
teams

['Sunrisers Hyderabad',
 'Mumbai Indians',
 'Chennai Super Kings',
 'Royal Challengers Bangalore',
 'Kings XI Punjab',
 'Delhi Capitals',
 'Kolkata Knight Riders',
 'Rajasthan Royals']

In [ ]:
match_df =  match_df[match_df['team1'].isin(teams)]
match_df = match_df[match_df['team2'].isin(teams)]

In [ ]:
match_df['team1'].unique()

array(['Sunrisers Hyderabad', 'Royal Challengers Bangalore',
       'Kolkata Knight Riders', 'Kings XI Punjab', 'Delhi Capitals',
       'Mumbai Indians', 'Chennai Super Kings', 'Rajasthan Royals'],
      dtype=object)

In [ ]:
match_df.shape

(641, 20)

In [ ]:
match_df['dl_applied'].value_counts()

dl_applied
0    626
1     15
Name: count, dtype: int64

In [ ]:
match_df = match_df[match_df['dl_applied'] == 0]

In [ ]:
match_df.head()

,id,Season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3,match_id,target
0,1,IPL-2017,Hyderabad,05-04-2017,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Sunrisers Hyderabad,35,0,Yuvraj Singh,"Rajiv Gandhi International Stadium, Uppal",AY Dandekar,NJ Llong,NaN,1,208
4,5,IPL-2017,Bangalore,08-04-2017,Royal Challengers Bangalore,Delhi Capitals,Royal Challengers Bangalore,bat,normal,0,Royal Challengers Bangalore,15,0,KM Jadhav,M Chinnaswamy Stadium,NaN,NaN,NaN,5,158
6,7,IPL-2017,Mumbai,09-04-2017,Kolkata Knight Riders,Mumbai Indians,Mumbai Indians,field,normal,0,Mumbai Indians,0,4,N Rana,Wankhede Stadium,Nitin Menon,CK Nandan,NaN,7,179
7,8,IPL-2017,Indore,10-04-2017,Royal Challengers Bangalore,Kings XI Punjab,Royal Challengers Bangalore,bat,normal,0,Kings XI Punjab,0,8,AR Patel,Holkar Cricket Stadium,AK Chaudhary,C Shamshuddin,NaN,8,149
9,10,IPL-2017,Mumbai,12-04-2017,Sunrisers Hyderabad,Mumbai Indians,Mumbai Indians,field,normal,0,Mumbai Indians,0,4,JJ Bumrah,Wankhede Stadium,Nitin Menon,CK Nandan,NaN,10,159


In [ ]:
match_df = match_df[['match_id', 'city', 'winner', 'target']]

In [ ]:
deliveries_df = match_df.merge(df_deliveries, on = 'match_id')

In [ ]:
deliveries_df = deliveries_df[deliveries_df['inning'] == 2]

In [ ]:
deliveries_df.head()

,match_id,city,winner,target,inning,batting_team,bowling_team,over,ball,batsman,...,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed,dismissal_kind,fielder
125,1,Hyderabad,Sunrisers Hyderabad,208,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,1,CH Gayle,...,0,0,0,0,1,0,1,NaN,NaN,NaN
126,1,Hyderabad,Sunrisers Hyderabad,208,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,2,Mandeep Singh,...,0,0,0,0,0,0,0,NaN,NaN,NaN
127,1,Hyderabad,Sunrisers Hyderabad,208,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,3,Mandeep Singh,...,0,0,0,0,0,0,0,NaN,NaN,NaN
128,1,Hyderabad,Sunrisers Hyderabad,208,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,4,Mandeep Singh,...,0,0,0,0,2,0,2,NaN,NaN,NaN
129,1,Hyderabad,Sunrisers Hyderabad,208,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,5,Mandeep Singh,...,0,0,0,0,4,0,4,NaN,NaN,NaN


In [ ]:
deliveries_df['current_score'] = deliveries_df.groupby('match_id')['total_runs'].cumsum()

In [ ]:
deliveries_df['runs_left'] = deliveries_df['target'] - deliveries_df['current_score']

In [ ]:
deliveries_df['balls_left'] = 126 - (deliveries_df['over']*6 + deliveries_df['ball'])

In [ ]:
deliveries_df['player_dismissed'] = deliveries_df['player_dismissed'].fillna('0')
deliveries_df['player_dismissed'] = deliveries_df['player_dismissed'].apply(lambda x: x if x == '0' else '1')
deliveries_df['player_dismissed'] = deliveries_df['player_dismissed'].astype(int)

In [ ]:
wickets = deliveries_df.groupby('match_id')['player_dismissed'].cumsum()

In [ ]:
deliveries_df['wickets'] = 10 - wickets

In [ ]:
deliveries_df['crr'] = (deliveries_df['current_score']/ (120-deliveries_df['balls_left'])) * 6

In [ ]:
deliveries_df['rrr'] =  (deliveries_df['runs_left'] / deliveries_df['balls_left']) * 6

In [ ]:
deliveries_df

,match_id,city,winner,target,inning,batting_team,bowling_team,over,ball,batsman,...,total_runs,player_dismissed,dismissal_kind,fielder,current_score,runs_left,balls_left,wickets,crr,rrr
125,1,Hyderabad,Sunrisers Hyderabad,208,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,1,CH Gayle,...,1,0,NaN,NaN,1,207,119,10,6.000000,10.436975
126,1,Hyderabad,Sunrisers Hyderabad,208,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,2,Mandeep Singh,...,0,0,NaN,NaN,1,207,118,10,3.000000,10.525424
127,1,Hyderabad,Sunrisers Hyderabad,208,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,3,Mandeep Singh,...,0,0,NaN,NaN,1,207,117,10,2.000000,10.615385
128,1,Hyderabad,Sunrisers Hyderabad,208,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,4,Mandeep Singh,...,2,0,NaN,NaN,3,205,116,10,4.500000,10.603448
129,1,Hyderabad,Sunrisers Hyderabad,208,2,Royal Challengers Bangalore,Sunrisers Hyderabad,1,5,Mandeep Singh,...,4,0,NaN,NaN,7,201,115,10,8.400000,10.486957
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149573,11415,Hyderabad,Mumbai Indians,153,2,Chennai Super Kings,Mumbai Indians,20,2,RA Jadeja,...,1,0,NaN,NaN,152,1,4,5,7.862069,1.500000
149574,11415,Hyderabad,Mumbai Indians,153,2,Chennai Super Kings,Mumbai Indians,20,3,SR Watson,...,2,0,NaN,NaN,154,-1,3,5,7.897436,-2.000000
149575,11415,Hyderabad,Mumbai Indians,153,2,Chennai Super Kings,Mumbai Indians,20,4,SR Watson,...,1,1,run out,KH Pandya,155,-2,2,4,7.881356,-6.000000
149576,11415,Hyderabad,Mumbai Indians,153,2,Chennai Super Kings,Mumbai Indians,20,5,SN Thakur,...,2,0,NaN,NaN,157,-4,1,4,7.915966,-24.000000


In [ ]:
def results(row):
    return 1 if row['batting_team'] == row['winner'] else 0

In [ ]:
deliveries_df['result']  = deliveries_df.apply(results, axis = 1)

In [ ]:
final_df = deliveries_df[['batting_team', 'bowling_team', 'city', 'target', 'balls_left', 'wickets', 'runs_left', 'crr', 'rrr', 'result']]

In [ ]:
final_df = final_df.sample(final_df.shape[0])

In [ ]:
final_df

,batting_team,bowling_team,city,target,balls_left,wickets,runs_left,crr,rrr,result
7896,Royal Challengers Bangalore,Mumbai Indians,Mumbai,166,81,9,121,6.923077,8.962963,1
54284,Delhi Daredevils,Deccan Chargers,Hyderabad,176,36,6,47,9.214286,7.833333,1
66112,Kings XI Punjab,Royal Challengers Bangalore,Bangalore,159,108,10,142,8.500000,7.888889,1
88688,Sunrisers Hyderabad,Kings XI Punjab,Sharjah,194,46,5,104,7.297297,13.565217,0
111130,Mumbai Indians,Royal Challengers Bangalore,Mumbai,236,41,8,110,9.569620,16.097561,0
...,...,...,...,...,...,...,...,...,...,...
51792,Kings XI Punjab,Delhi Daredevils,Delhi,232,86,9,190,7.411765,13.255814,0
148552,Mumbai Indians,Kolkata Knight Riders,Mumbai,144,71,9,78,8.081633,6.591549,1
123348,Delhi Daredevils,Mumbai Indians,Mumbai,205,64,9,110,10.178571,10.312500,1
122413,Sunrisers Hyderabad,Rajasthan Royals,Hyderabad,134,50,9,32,8.742857,3.840000,1


In [ ]:
final_df.dropna(inplace = True)

In [ ]:
final_df = final_df[final_df['balls_left'] != 0]

In [ ]:
X = final_df.iloc[:,: -1]
y = final_df.iloc[:,-1]
from sklearn.model_selection import train_test_split
X_train, X_test,y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 10)

In [ ]:
X_train

,batting_team,bowling_team,city,target,balls_left,wickets,runs_left,crr,rrr
45633,Deccan Chargers,Kings XI Punjab,Dharamsala,175,51,8,70,9.130435,8.235294
118357,Delhi Daredevils,Kings XI Punjab,Chandigarh,182,5,5,25,8.191304,30.000000
106281,Sunrisers Hyderabad,Mumbai Indians,Mumbai,158,78,8,101,8.142857,7.769231
21035,Royal Challengers Bangalore,Chennai Super Kings,Port Elizabeth,180,105,9,158,8.800000,9.028571
116548,Sunrisers Hyderabad,Kings XI Punjab,Hyderabad,144,116,10,139,7.500000,7.189655
...,...,...,...,...,...,...,...,...,...
92978,Royal Challengers Bangalore,Mumbai Indians,Mumbai,188,46,8,71,9.486486,9.260870
45873,Royal Challengers Bangalore,Mumbai Indians,Bangalore,192,56,6,120,6.750000,12.857143
35236,Kolkata Knight Riders,Chennai Super Kings,Kolkata,165,62,5,102,6.517241,9.870968
29832,Mumbai Indians,Rajasthan Royals,Durban,146,60,7,92,5.400000,9.200000


In [ ]:
X_train.describe()

,target,balls_left,wickets,runs_left,crr,rrr
count,57073.000000,57073.000000,57073.000000,57073.000000,57073.000000,57073.000000
mean,166.697913,62.966902,7.560650,93.688662,7.446748,10.550627
std,29.320436,33.250888,2.121305,49.928400,2.278455,13.595758
min,66.000000,-2.000000,0.000000,-15.000000,0.000000,-516.000000
25%,148.000000,35.000000,6.000000,54.000000,6.260870,7.266667
50%,166.000000,64.000000,8.000000,93.000000,7.500000,8.974359
75%,186.000000,92.000000,9.000000,131.000000,8.700000,11.027027
max,251.000000,119.000000,10.000000,250.000000,42.000000,588.000000


In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
trf = ColumnTransformer([
    ('trf', OneHotEncoder(drop = 'first'), ['batting_team', 'bowling_team', 'city'])
],  remainder = 'passthrough')

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

In [ ]:
lgr = LogisticRegression(solver = 'liblinear')
pipe= Pipeline(steps=[('step1', trf),
            ('step2', lgr)]
)


In [ ]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('step1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('trf',
                                                  OneHotEncoder(drop='first'),
                                                  ['batting_team',
                                                   'bowling_team', 'city'])])),
                ('step2', LogisticRegression(solver='liblinear'))])

In [ ]:
y_pred = pipe.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.8067839372065316

In [ ]:
data = {
    'batting_team' : ['Kings XI Punjab'],
    'bowling_team' : ['Sunrisers Hyderabad'],
    'city' : ['Chandigarh'],
    'target' : [167],
    'balls_left' : [90],
    'wickets': [7],
    'runs_left' : [120],
    'crr' : [7.4],
    'rrr' : [8]
}
temp_df = pd.DataFrame(data)

In [ ]:
pipe.predict_proba(temp_df)

array([[0.64023235, 0.35976765]])

In [ ]:
jb.dump(pipe, 'IPL_Win_Predictor.plk')

['IPL_Win_Predictor.plk']

In [ ]:
IPL_WinWiz = jb.load('/content/IPL_Win_Predictor.plk')

In [ ]:
IPL_WinWiz.predict_proba(temp_df)

array([[0.64023235, 0.35976765]])

In [ ]:
def predict(batting_team, bowling_team, city, target, balls_left, wickets, runs_scored):
    new_df = pd.DataFrame({
        'batting_team' : [batting_team],
        'bowling_team' : [bowling_team],
        'city' : [city],
        'target' : [target],
        'balls_left': [balls_left],
        'wickets' : [wickets],
        'runs_left' : [(target- runs_scored)],
        'crr' : [(runs_scored)*6/(120 -balls_left)],
        'rrr' : [(target-runs_scored)*6/balls_left]
    })
    prediction = IPL_WinWiz.predict_proba(new_df)
    bowling_prob = prediction[0][0] * 100
    batting_prob = prediction[0][1] * 100

    result =  f"""
    <div style="font-size: 20px; font-weight: bold;">
        <p><span style="color: blue;">{batting_team}</span>: <b>{batting_prob}%</b></p>
        <p><span style="color: red;">{bowling_team}</span>: <b>{bowling_prob}%</b></p>
    </div>
    """

    return result


In [ ]:
predict('Mumbai Indians', 'Sunrisers Hyderabad', 'Hyderabad', 158, 52, 9, 77)

'\n    <div style="font-size: 20px; font-weight: bold;">\n        <p><span style="color: blue;">Mumbai Indians</span>: <b>80.93863190392871%</b></p>\n        <p><span style="color: red;">Sunrisers Hyderabad</span>: <b>19.061368096071284%</b></p>\n    </div>\n    '